In [ ]:
#pip install qiskit

In [ ]:
#pip install pylatexenc

In [ ]:
#initialization
import matplotlib.pyplot as plt
import numpy as np
import math
from qiskit.circuit.library import MCMT

# importing Qiskit

from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, Aer
from qiskit.providers.ibmq import least_busy
from math import pi
# import basic plot tools
from qiskit.visualization import plot_histogram
from qiskit.tools.monitor import job_monitor
from qiskit.compiler import transpile

In [ ]:
def initialize_s(qc, qubits):
    """Apply a H-gate to 'qubits' in qc"""
    for q in qubits:
        qc.h(q)
    return qc

def diffuser(nqubits):
    qc = QuantumCircuit(nqubits)
    #Uniform Superposition
    for qubit in range(nqubits):
        qc.h(qubit)
    
    for qubit in range(nqubits):
        qc.x(qubit)
    # Do multi-controlled-Z gate
    qc.h(nqubits-1)
    qc.mct(list(range(nqubits-1)), nqubits-1)  # multi-controlled-toffoli
    qc.h(nqubits-1)
    # Apply transformation |11..1> -> |00..0>
    for qubit in range(nqubits):
        qc.x(qubit)
    # Apply transformation |00..0> -> |s>
    for qubit in range(nqubits):
        qc.h(qubit)
    # We will return the diffuser as a gate
    U_s = qc.to_gate()
    U_s.name = "U$_s$"
    return U_s


In [ ]:
n = 2
qc = QuantumCircuit(n)

qc.cz(0,1)

oracle_ex3 = qc.to_gate()
oracle_ex3.name = "U$_\omega$"

grover_circuit = QuantumCircuit(n)
grover_circuit = initialize_s(grover_circuit, [0,1,2,3,4])
grover_circuit.append(oracle_ex3, [0,1,2,3,4])
grover_circuit.append(diffuser(n), [0,1,2,3,4])
grover_circuit.measure_all()

qasm_sim = Aer.get_backend('qasm_simulator')
transpiled_grover_circuit = transpile(grover_circuit, qasm_sim)
results = qasm_sim.run(transpiled_grover_circuit).result()
counts = results.get_counts()
plot_histogram(counts)

In [ ]:
grover_circuit.draw()